In [1]:
from app import App

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
SYMBOLS = ["BTCUSDT", "ETHUSDT", "QNTUSDT"]
INTERVAL = "1d"

backtest = App(symbols = SYMBOLS, interval = INTERVAL, start="2022-10-01", end = "2022-11-10")
backtest.run()

In [3]:
backtest.results()

 ==== BTCUSDT ==== 
                  LS  LS_reduit      long     short
total_PL    0.909568   0.909568  0.960472  0.947001
nb_trades   0.000000   0.000000  0.000000  0.000000
min_ret    -0.045283  -0.039716 -0.039716  0.000000
quartile_1 -0.042937  -0.039392 -0.039415  0.000000
median     -0.040591  -0.039068 -0.039114  0.000000
mean        0.997798   0.994748  0.995975  0.000000
quartile_3 -0.038245  -0.038745 -0.038814  0.000000
max_ret     0.038818   0.066984  0.066984  0.000000
drawdown   -0.095420  -0.095420 -0.075458 -0.060454
skew        0.284630   1.501180  1.438942  0.000000
kurt        2.077895   3.860778  2.000109  0.000000
VaR_hist    0.044814   0.039651  0.039656  0.032077
VaR-gauss   0.049117   0.077670  0.000000  0.000000
CVaR        0.002100   0.009247  0.011515  0.006696
 ==== ETHUSDT ==== 
                  LS  LS_reduit      long     short
total_PL    1.016632   1.016632  1.084000  0.937852
nb_trades   0.000000   0.000000  0.000000  0.000000
min_ret    -0.069971  -0

In [4]:
backtest.plot("BTCUSDT")

In [5]:
backtest.plot("ETHUSDT")

In [6]:
backtest.plot("QNTUSDT")

In [7]:
backtest.plot_returns("BTCUSDT")

ValueError: 
The 'specs' argument to make_subplots must be a 2D list of dictionaries with dimensions (2 x 1).
    Received value of type <class 'list'>: [[{'secondary_y': True}], [{'secondary_y': True}], [{'secondary_y': True}]]

In [ ]:
backtest.data['BTCUSDT']

In [ ]:
df = backtest.dataLS.copy()
df_r = backtest.LS_reduit

In [ ]:
import pandas as pd
import numpy as np

data = pd.DataFrame(index = range(1, 31+1))

In [ ]:
df_nov = df_r.loc[df_r.date.dt.month == 11][['date', 'returns']]
df_nov['date'] = df_nov.date.dt.day
df_nov = df_nov.groupby('date').sum()
df_nov.rename(columns = {'returns': 'november'}, inplace = True)

In [ ]:
df_dec = df_r.loc[df_r.date.dt.month == 12][['date', 'returns']]
df_dec['date'] = df_dec.date.dt.day
df_dec = df_dec.groupby('date').sum()
df_dec.rename(columns = {'returns': 'december'}, inplace = True)

In [ ]:
dd = pd.concat([data, df_nov, df_dec], axis=1)
dd.T

In [ ]:
import plotly.express as px

px.imshow(dd.T)

# Compare

In [ ]:
backtest.data_1.head(25)[['return', 'log_return', 'momentum']]

In [ ]:
col = ['price', 'status', 'amount', 'quantity', 'value', 'returns', 'cum_ret_per_trade', 'cum_ret', 'price_cum_ret']
col_r = ['price', 'status', 'amount', 'quantity', 'value', 'returns', 'cum_ret', 'price_cum_ret']

data_LS1 = backtest.output["BTCUSDT"][0][col]
LS_reduit = backtest.output["BTCUSDT"][1][col_r]


import sqlalchemy
import pandas as pd

path = "C:/Users/cc/Desktop/CedAlgo/AlgoTrading/data/trade/"
engine = sqlalchemy.create_engine('sqlite:///'+path+'BTCUSDT_LS.db')

data_LS2 = pd.read_sql('DataLS', engine)[['date']+col]
LS_reduit2 = pd.read_sql('LS_reduit', engine)[col_r]

In [ ]:
data_LS1.head(25)

In [ ]:
data_LS2.head(25)

In [ ]:
fig.update_layout(    
    {    
        'xaxis':{'range': [0, 1], 'dtick': 0.2,},
        'yaxis':{'range': [0, 1], 'dtick': 0.2,},
        'xaxis2':{'range': [0, 1], 'dtick': 0.2,},
        'yaxis2':{'range': [0, 1], 'dtick': 0.2,},
        'xaxis3':{'range': [0, 1], 'dtick': 0.2,},
        'yaxis3':{'range': [0, 1], 'dtick': 0.2,},
        'xaxis4':{'range': [0, 1], 'dtick': 0.2,},
        'yaxis4':{'range': [0, 1], 'dtick': 0.2,},
        'height':800,
        'width':800,
        'template':'plotly_white'
    }
)

In [ ]:
fig.update_layout(xaxis1=dict(rangeslider=dict(visible=False)),
                  xaxis2=dict(rangeslider=dict(visible=False)),
                  xaxis3=dict(rangeslider=dict(visible=False)),
                  xaxis4=dict(rangeslider=dict(visible=False)),
                  xaxis5=dict(rangeslider=dict(visible=False)),
                  xaxis6=dict(rangeslider=dict(visible=False)),
                  xaxis7=dict(rangeslider=dict(visible=False)),
                  xaxis8=dict(rangeslider=dict(visible=False)),
                  xaxis9=dict(rangeslider=dict(visible=False)),
                  xaxis10=dict(rangeslider=dict(visible=False)),
                  xaxis11=dict(rangeslider=dict(visible=False)),
                  xaxis12=dict(rangeslider=dict(visible=False)),
                  xaxis13=dict(rangeslider=dict(visible=False)),
                  xaxis14=dict(rangeslider=dict(visible=False)),
                 )

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("Plot 1", "Plot 2", "Plot 3", "Plot 4")
)

# Add traces
fig.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6]), row=1, col=1)
fig.add_trace(go.Scatter(x=[20, 30, 40], y=[50, 60, 70]), row=1, col=2)
fig.add_trace(go.Scatter(x=[300, 400, 500], y=[600, 700, 800]), row=2, col=1)
fig.add_trace(go.Scatter(x=[4000, 5000, 6000], y=[7000, 8000, 9000]), row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="xaxis 1 title", row=1, col=1)
fig.update_xaxes(title_text="xaxis 2 title", range=[10, 50], row=1, col=2)
fig.update_xaxes(title_text="xaxis 3 title", showgrid=False, row=2, col=1)
fig.update_xaxes(title_text="xaxis 4 title", type="log", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="yaxis 1 title", row=1, col=1)
fig.update_yaxes(title_text="yaxis 2 title", range=[40, 80], row=1, col=2)
fig.update_yaxes(title_text="yaxis 3 title", showgrid=False, row=2, col=1)
fig.update_yaxes(title_text="yaxis 4 title", row=2, col=2)

# Update title and height
fig.update_layout(title_text="Customizing Subplot Axes", height=700)

fig.show()